In [1]:
import numpy as np
from PIL import Image
import cv2
import pandas as pd

In [2]:
image_basic_path:str='./计数/cells_image/'
nums_record:str='./计数/nums.txt'
nums_file=open(nums_record)
'''计数文件'''
nums_ndarray:np.ndarray=np.empty(shape=[200,2],dtype='object')
'''将图片的文件名和所记录的数目整合在一个数组里面，200行2列，第一列是图片名，第二列是细胞数量'''
num_line:str=nums_file.readline()
index:int=0
while num_line:
    str_list:str=num_line.split(' ',1)
    nums_ndarray[index,0]=str_list[0]
    nums_ndarray[index,1]=str_list[1][:-1]
    index=index+1
    num_line=nums_file.readline()

print(nums_ndarray)

[['001dots.png' '135']
 ['002dots.png' '233']
 ['003dots.png' '82']
 ['004dots.png' '86']
 ['005dots.png' '96']
 ['006dots.png' '143']
 ['007dots.png' '169']
 ['008dots.png' '122']
 ['009dots.png' '168']
 ['010dots.png' '115']
 ['011dots.png' '117']
 ['012dots.png' '301']
 ['013dots.png' '78']
 ['014dots.png' '146']
 ['015dots.png' '155']
 ['016dots.png' '199']
 ['017dots.png' '223']
 ['018dots.png' '195']
 ['019dots.png' '197']
 ['020dots.png' '183']
 ['021dots.png' '158']
 ['022dots.png' '200']
 ['023dots.png' '168']
 ['024dots.png' '241']
 ['025dots.png' '315']
 ['026dots.png' '112']
 ['027dots.png' '196']
 ['028dots.png' '111']
 ['029dots.png' '255']
 ['030dots.png' '97']
 ['031dots.png' '190']
 ['032dots.png' '197']
 ['033dots.png' '136']
 ['034dots.png' '137']
 ['035dots.png' '142']
 ['036dots.png' '238']
 ['037dots.png' '212']
 ['038dots.png' '198']
 ['039dots.png' '228']
 ['040dots.png' '178']
 ['041dots.png' '223']
 ['042dots.png' '280']
 ['043dots.png' '126']
 ['044dots.png' 

先用单幅图来进行测试（已经将测试过程中使用的代码改成函数）

In [12]:
line:np.ndarray=nums_ndarray[:,0]
'''图片名称列'''
image=Image.open(image_basic_path+line[0])
'''第一幅图像'''
pixel_num:int=image.size[0]*image.size[1]
'''像素点个数'''
print(f"像素点个数为：{pixel_num}")
r,g,b=image.split()
print(type(image))
print(np.size(image))
image_array:np.ndarray=np.array(b)#保留蓝色通道
# cv2.imshow('image_array_1',image_array)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
image_array = cv2.GaussianBlur(image_array,(5,5),0)#滤波去噪
# cv2.imshow('image_array_2',image_array)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

像素点个数为：65536
<class 'PIL.PngImagePlugin.PngImageFile'>
(256, 256)


In [4]:
def HarrisCorner(image_array:np.ndarray)->int:
    '''Harris角点特征提取'''
    corners_mark = cv2.cornerHarris(image_array, blockSize=5,ksize=3,k=0.1)
    corners_mark=cv2.erode(corners_mark,None)#腐蚀
    corners_mark=cv2.dilate(corners_mark,None)#膨胀
    # print(np.size(corner_image))
    # print(np.size(image))


    corners_image=image_array.copy()#注意这里一定要深拷贝
    corners_image[corners_mark>0.005*corners_mark.max()]=255
    '''角点坐标数组'''
    # cv2.imshow('corners_image',corners_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    count_array=np.zeros(np.size(image))
    '''计数使用的数组'''
    count_array[corners_mark>0.005*corners_mark.max()]=1
    # print(count_array[:,0])
    corner_num:int=int(np.sum(count_array))
    # print(corner_num)
    return corner_num

In [5]:
def getContours(image_array:np.ndarray)->tuple:
    '''获取轮廓'''
    # thresh, image_thresh = cv2.threshold(image_array, 127, 255, 0)
    image_thresh = cv2.adaptiveThreshold(image_array,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,13,-10)
    image_thresh=cv2.erode(image_thresh,None)#腐蚀
    image_thresh=cv2.dilate(image_thresh,None)#膨胀
    contours, hierarchy=cv2.findContours(image_thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    contours:tuple
    '''轮廓'''
    # hierarchy输出各个轮廓的继承关系
    # contours_image=cv2.drawContours(image_thresh,contours,-1,(127,255,127),2)
    # cv2.imshow('contours_image',contours_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return contours


In [6]:
def getContoursNumPerimeterAndArea(image_array:np.ndarray)->tuple:
    '''获取轮廓条数、长度和面积，其中返回的面积是轮廓圈出来的面积占总面积的比例'''
    contours:tuple=getContours(image_array)
    # print(len(contours))#轮廓数量
    # print(contours)
    #每一条轮廓是一个array。对于每一条轮廓由若干个点组成
    contours_num:int=len(contours)

    #计算所有轮廓周长
    # contour_len=cv2.arcLength(contours[0],closed=True)
    # print(contour_len)#第一条轮廓长
    contours_len:float=0.0
    '''轮廓总长度'''
    for contour in contours:
        contour_len=cv2.arcLength(contours[0],closed=True)
        '''每个轮廓的长度'''
        contours_len=contours_len+contour_len
    # print(contours_len)#轮廓总长

    #计算所有轮廓围成的面积
    contours_area:float=0.0
    for contour in contours:
        contour_area=cv2.contourArea(contour,oriented=False)
        contours_area=contours_area+contour_area
    # print(contours_area)#轮廓围成的面积
    contours_area_normalized:float=contours_area/pixel_num
    # print(contours_area_normalized)#归一化轮廓围成的面积
    return (contours_num,contours_len,contours_area_normalized)

In [7]:
def CannyEdgesPointsNum(image_array:np.ndarray)->int:
    '''canny边缘点的个数'''
    canny_low_threshold=30
    canny_ratio=3
    canny_kernel_size=3
    canny_edges:np.ndarray=cv2.Canny(image_array,canny_low_threshold,canny_low_threshold*canny_ratio,apertureSize=canny_kernel_size)

    # cv2.imshow('canny',canny_edges)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # np.set_printoptions(threshold=np.inf)
    # print(canny_edges[0:10,:])
    # print(type(canny_edges))
    # print(canny_edges.shape)
    #获得的数据不是有层次的，所以只能统计点的个数
    canny_edges_points_num=int(np.sum(canny_edges)/255)
    # print(canny_edges_points_num)
    return canny_edges_points_num


In [8]:
columns_index:list=['HarrisCorner','ContoursNum','ContoursPerimeter','ContoursArea','CannyEdges','CellNum']
features=pd.DataFrame(columns=columns_index,dtype='object')
for image_and_cell_num in nums_ndarray:
    image_and_cell_num:np.ndarray
    image_name:str=image_and_cell_num[0]
    cells_num:int=int(image_and_cell_num[1])
    image=Image.open(image_basic_path+image_name)
    '''第一幅图像'''
    pixel_num:int=image.size[0]*image.size[1]
    '''像素点个数'''
    # print(pixel_num)
    r,g,b=image.split()
    image_array:np.ndarray=np.array(b)#保留蓝色通道
    image_array = cv2.GaussianBlur(image_array,(5,5),0)#滤波去噪
    corner_num=HarrisCorner(image_array)
    contours_num,contours_perimeter,contours_area=getContoursNumPerimeterAndArea(image_array)
    contours_num:int
    contours_perimeter:float
    contours_area:float
    canny_edges_points_num:int=CannyEdgesPointsNum(image_array)
    features_line=pd.Series([corner_num,contours_num,contours_perimeter,contours_area,canny_edges_points_num,cells_num],index=columns_index,dtype='object')
    # print(features_line)
    # print(features_line)
    features=pd.concat([features,features_line.to_frame().T],ignore_index=True,axis=0)
print('得到的特征的数值为：')
print(features)



得到的特征的数值为：
    HarrisCorner ContoursNum ContoursPerimeter ContoursArea CannyEdges CellNum
0           5750          92       2880.861164     0.120544       3726     135
1           8495         114       2858.101717     0.177223       5636     233
2           3461          59       1861.824586     0.076401       2328      82
3           3697          61       1392.534052     0.075897       2331      86
4           4120          71       2064.045806     0.089516       2774      96
..           ...         ...               ...          ...        ...     ...
195         4017          71       1922.045806     0.084335       2577      84
196         8914         126       3441.527246     0.178223       5794     218
197         3931          64       1551.528999     0.076118       2373      79
198         6211         102       1308.499562     0.132088       4161     153
199         5136          82       4315.448162     0.113106       3495     124

[200 rows x 6 columns]


In [9]:
features=features.astype('float32')
#划分数据集和测试集
train_num=100
test_num=200-train_num
train_set:pd.DataFrame=features.loc[0:train_num-1,:]
test_set:pd.DataFrame=features.loc[train_num:200,:]

x_train:pd.DataFrame=train_set.loc[:,'HarrisCorner':'CannyEdges']
y_train:pd.DataFrame=train_set.loc[:,'CellNum']
# print(x_train.to_numpy().shape)
x_train:np.ndarray=np.asmatrix(x_train.to_numpy().copy())
# print(x_train.shape)
#注意前面还要加一个维度（对应b0）
x_train=np.insert(arr=x_train,obj=0,values=1,axis=1)
y_train:np.ndarray=np.asmatrix(y_train.to_numpy().copy())
x_train=x_train.T#每一行是一个特征，每一列是一份数据
y_train=y_train.T
# beta:np.ndarray=np.random.randn(6,1)
# print(x_train.shape)
# print(y_train.shape)

beta_hut=((x_train@x_train.T).I)@x_train@y_train
# print(beta_hut.shape)
print("估计的各个参数为")
print(beta_hut)


估计的各个参数为
[[-2.0803898e+01]
 [ 1.1102706e-03]
 [-8.1074357e-01]
 [-1.1655502e-05]
 [ 1.2675491e+03]
 [ 1.9595757e-02]]


In [10]:
x_test:pd.DataFrame=test_set.loc[:,'HarrisCorner':'CannyEdges']
y_test:pd.DataFrame=test_set.loc[:,'CellNum']
x_test:np.ndarray=np.asmatrix(x_test.to_numpy().copy())
# print(x_train.shape)
#注意前面还要加一个维度（对应b0）
x_test=np.insert(arr=x_test,obj=0,values=1,axis=1)
y_test:np.ndarray=np.asmatrix(y_test.to_numpy().copy())
x_test=x_test.T#每一行是一个特征，每一列是一份数据
y_test=y_test.T
y_sim:np.ndarray=x_test.T@beta_hut
# print(y_sim)
y_difference:np.ndarray=y_sim-y_test
y_difference=np.maximum(y_difference,-y_difference)
print('得到的和实际值的差')
print(y_difference.T)
print('取绝对值之后误差的平均值为')
print(np.mean(y_difference))
print('方差为')
print(y_difference.T@y_difference/test_num)

得到的和实际值的差
[[ 1.9802551   3.7909698   4.868683    1.8350983   9.171265    2.7404327
   1.5470428   7.884796    2.4114532   3.0551147   5.3714294   8.054871
   3.1778717   3.1191635   2.0220795   1.5046082   3.3765564   9.968979
   4.4338684   0.8067322   0.719574    2.8284607   4.8678894   1.3103638
   0.24656677  0.37153625  2.8838806   3.3743591   6.0464478   9.217926
   1.6685486   0.06274414  1.5948105   2.8739624   7.6906433   5.7682495
   1.2388611   8.569168    5.8698425   3.9786606   9.386078    3.9540634
  16.665771    6.3063965   5.71257     2.1459427   2.3667603   0.6270752
  11.950897   10.147522    0.27383423  1.8653259   3.734436    2.6347656
   4.795311   15.918915    6.053589    2.584381    2.2861328   6.4261475
   1.3187714  10.981995    7.301483    5.480194    5.4609375   2.180664
   4.3552322   0.95314026  9.752533    4.6986084   0.44683838  2.9656677
   5.0200043   4.2384644   2.9171448   0.21652985  3.4134064   1.7176666
   2.5711365   3.5711365   7.945877    5.9900

In [11]:
# beta:np.ndarray=np.random.randn(pixel_num,1)
# y_train:np.ndarray=nums_nparray[0:100,1].astype('int16')
# A=images_array_train.T@y_train
# B=images_array_train.T@images_array_train
# B=B.astype('int16')
# beta_hut=A/np.linalg.det(B)
#TODO 这个表达式有问题。更改表达式之后应该不会出现内存不够的问题

# Unable to allocate 32.0 GiB for an array with shape (65536, 65536) and data type float64